#**Análisis de sentimientos y clasificación de emociones en español**

Este notebook describe el procedimiento para llevar a cabo un análisis automático de sentimientos (AS) y emociones (CE) en interacciones habladas en idioma español. El análisis se basa en la transcripción generada a partir de archivos de audio o video, y permite identificar tanto los sentimientos como las emociones expresadas por los participantes y durante toda la interacción.

##Configuración inicial

A continuación, se detallan las librerías principales que se utilizarán y su propósito dentro del flujo de trabajo.

### Instalación de WhisperX

**Whisper** es una librería de OpenAI para la transcripción de texto desde audio o video.

[**WhisperX**](https://github.com/m-bain/whisperX) es una extensión de Whisper que se enfoca en mejorar la precisión de la transcripción utilizando un conjunto de modelos y técnicas adicionales. Esto incluye la alineación de la transcripción con el audio, la diarización para separar a los diferentes hablantes y la traducción a otros idiomas.


In [ ]:
# Instalación completa de WhisperX y dependencias necesarias para GPU
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu126
!pip install git+https://github.com/m-bain/whisperx.git@v3.3.1 -q
!apt-get update
!apt-get install libcudnn8=8.9.2.26-1+cuda12.1
!apt-get install libcudnn8-dev=8.9.2.26-1+cuda12.1

# Activar cálculos TF32 para mejorar rendimiento en GPU
!python -c "import torch; torch.backends.cuda.matmul.allow_tf32 = True; torch.backends.cudnn.allow_tf32 = True"

# Motor de inferencia rápido utilizado por WhisperX
!pip install ctranslate2==4.4.0

# Librería para diarización (identificación de hablantes)
!pip install pyannote.audio==0.0.1

# Traductor automático multilingüe (usado para traducir la transcripción)
!pip install --upgrade translators

### Instalación de PySentimiento

[**PySentimiento**](https://huggingface.co/pysentimiento/robertuito-sentiment-analysis) es una librería que utiliza modelos pre-entrenados de transformers para distintas tareas de SocialNLP. Usa como modelos bases a BETO y RoBERTuito en Español, BERTweet en inglés, y otros modelos similares en italiano y portugués.

Funcionalidades de Pysentimiento:

  * *Análisis de sentimientos:* Detecta si el sentimiento en un texto es positivo, negativo o neutro.
  * *Detección de emociones:* Identifica emociones específicas como alegría, tristeza, ira, etc.
  * *Clasificación de opiniones:* Permite categorizar textos según opiniones o puntos de vista.

In [ ]:
!pip install pysentimiento

###roberta-base-go_emotions

[**roberta-base-go_emotions**](https://huggingface.co/SamLowe/roberta-base-go_emotions) es un modelo de clasificación de emociones basado en **RoBERTa**, entrenado sobre el conjunto de datos **GoEmotions** de Google.

Este modelo permite detectar múltiples emociones simultáneamente en un texto (clasificación multi-etiqueta), y es especialmente útil para tareas de análisis emocional fino en **inglés**.

#### Características principales:
- Basado en **RoBERTa-base**, un modelo robusto para comprensión del lenguaje.
- Entrenado sobre más de 58 mil frases con etiquetas de **27 emociones + neutral**.
- Permite obtener una distribución de emociones como: *joy*, *anger*, *sadness*, *surprise*, *fear*, *disgust*, entre otras.
- Soporta análisis multilabel: un mismo texto puede transmitir varias emociones a la vez.

Este modelo complementa el análisis en español realizado con `pysentimiento`, y es útil para reevaluar o comparar resultados en textos traducidos al inglés.



In [ ]:
from transformers import pipeline
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

### Pydub

**Pydub** es una biblioteca de Python que facilita la manipulación de archivos de audio de forma simple e intuitiva.

Se utiliza principalmente para tareas de procesamiento y edición de audio, sin necesidad de trabajar a bajo nivel con señales o buffers.

Funcionalidades destacadas:
- Carga y exportación de archivos en múltiples formatos (WAV, MP3, etc.).
- Recorte de fragmentos de audio por tiempo.
- Unión o superposición de varios audios.
- Cambio de volumen, velocidad, canal o frecuencia de muestreo.
- Conversión de formatos.

En este proyecto, `pydub` se utiliza principalmente para obtener fragmentos de audio asociados a cada hablante, lo que facilita su posterior etiquetado manual.

In [ ]:
!pip install pydub

###Deep-Translator

**Deep-Translator** es una biblioteca que facilita la traducción automática utilizando distintos motores de traducción.

En este proyecto se eligió utilizar **GoogleTranslator** por las siguientes ventajas:

- No impone restricciones en la cantidad de caracteres a traducir.
- No requiere una clave de API (*API-Key*), lo que simplifica su uso y configuración.
- Proporciona traducciones rápidas y precisas para diversos idiomas.



In [ ]:
!pip install -U deep-translator

In [ ]:
from deep_translator import (GoogleTranslator,
                             single_detection,
                             batch_detection)

###NLTK: Tokenización y manejo de *stopwords*

**NLTK** (Natural Language Toolkit) es una biblioteca popular en Python para procesamiento de lenguaje natural.

Esta preparación es fundamental para limpiar y procesar textos antes de aplicar análisis de sentimientos o emociones.


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

###Imports

En este notebook utilizaremos diversas librerías y módulos para realizar el análisis automático de sentimientos. Algunas de estas librerías necesitan ser instaladas previamente, mientras que otras ya vienen preinstaladas en el entorno de Google Colab y sólo es necesario importarlas.


In [ ]:
# Procesamiento y transcripción de audio/video con aceleración en GPU
import torch
import whisperx

# Manejo y análisis de datos estructurados
import pandas as pd
import numpy as np

# Visualización gráfica y generación de gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Manipulación y comparación avanzada de texto
import string
import re

# Conteo eficiente de elementos (palabras, tokens)
from collections import Counter

# Gestión manual de memoria para optimizar recursos
import gc

# Analizador de sentimientos y emociones en español
from pysentimiento import create_analyzer

# Uso de modelos preentrenados para tareas de NLP
import transformers

# Generación de nubes de palabras para visualización
from wordcloud import WordCloud

# Interacción con archivos y sistema operativo en Colab
from google.colab import files
import os

# Manipulación de segmentos de audio
from pydub import AudioSegment

# Reproducción integrada de audio en notebooks
from IPython.display import Audio, display


In [ ]:
import locale
print(locale.getpreferredencoding())
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

###Token Hugging Face

Para usar esta herramienta, necesitas un token de acceso de Hugging Face. Puedes obtenerlo en:  [Token de HF](https://huggingface.co/docs/hub/security-tokens)

In [ ]:
HF_TOKEN = "" # Pega tu token aquí (ej. hf_XXXXXXXXXXXXXXXXXXXXXX)

##Carga y Preprocesamiento del Archivo de Audio/Video

Para realizar el análisis, primero necesitamos cargar el archivo que deseamos procesar. Para ello, ejecuta la siguiente celda y selecciona el archivo correspondiente.

**Importante:** Solo se puede cargar un archivo a la vez, y este debe tener una de las siguientes extensiones: **.mp3**, **.mp4** o **.wav**. De lo contrario, la carga no se realizará correctamente.





In [ ]:
# Subir archivo de audio o video
uploaded = files.upload()

if len(uploaded) == 1:
    filename = next(iter(uploaded.keys()))

    # Verifica la extensión del archivo
    valid_extensions = ['.wav', '.mp3', '.mp4']
    file_extension = os.path.splitext(filename)[1].lower()

    if file_extension in valid_extensions:
        print(f'\nArchivo válido subido: {filename}')
    else:
        print(f'\nError: La extensión del archivo {filename} no es válida. El archivo será eliminado.')
        os.remove(filename)  # Elimina el archivo subido si no es válido
else:
    print(f'\nError: Debes subir solo un archivo. Se eliminarán todos los archivos subidos.')
    # Eliminar todos los archivos subidos
    for filename in uploaded.keys():
        os.remove(filename)

## Transcripción de Texto con WhisperX

Cargar modelo WhisperX (ajustar a tus necesidades: tiny, base, medium, large)

WhisperX utiliza los mismos modelos de Whisper de OpenAI y agrega alineación precisa de palabras.

Los modelos disponibles son:

* **Tiny**: Muy rápido, bajo consumo de recursos, precisión limitada.
* **Base**: Rápido, buen rendimiento en inglés, adecuado para tareas simples.
* **Small**: Balance entre rapidez y precisión, mejor soporte multilingüe.
* **Medium**: Alta precisión en múltiples idiomas, más lento, ideal para audio ruidoso.
* **Large**: Máxima precisión, ideal para tareas exigentes, pero lento y pesado.

In [ ]:
device = "cuda"
batch_size = 8 # Reducir si hay poca memoria GPU.
compute_type = "float16" # Cambiar a "int 8" si hay poca GPU (posiblemente reduzca la presición)

In [ ]:
model = whisperx.load_model("large-v2", device, compute_type=compute_type, asr_options={'hotwords': []})

Ejecutamos la siguiente celda para realizar la transcripción y alineación de los segmentos

In [ ]:
# Transcribir el audio
audio = whisperx.load_audio(filename)
result = model.transcribe(audio, batch_size=batch_size)

# Alinear segmentos con un modelo de alineamiento adicional
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

#Guarda la transcripcion en un archivo .txt
transcription_text = "\n".join([segment['text'] for segment in result["segments"]])
with open('transcription.txt', 'w', encoding='utf-8') as file: # Modified line: explicitly use utf-8 encoding.
  file.write(transcription_text)

# Mostramos las marcas temporales y el texto alineado
for segmento in result["segments"]:
    print(f"[{segmento['start']} - {segmento['end']}] {segmento['text']}")

Si desea descargar la transcripción como un archivo de texto, ejecutar la siguiente celda.

In [ ]:
files.download('transcription.txt')

## Diarización

**Definición:** La diarización es el proceso de identificar y separar las voces de los diferentes hablantes en una grabación de audio o en un flujo de video.

**IMPORTANTE:**  Para realizar la diarización de manera precisa, es fundamental especificar correctamente la cantidad de hablantes presentes en el archivo de audio. Proporcionar un número incorrecto puede llevar a errores en la asignación de intervenciones por parte de la herramienta **WhisperX**, lo que afecta negativamente la calidad del análisis.

No obstante, en caso de que se desconozca la cantidad exacta de participantes, **WhisperX** puede estimarla automáticamente durante el proceso de análisis, sin interrumpir la ejecución.

Si conoce el número exacto de participantes, cambie el número y ejecute la siguiente línea

In [ ]:
numero_participantes = 2 #cambiar el número según corresponda

In [ ]:
#Asignar etiquetas de speaker (orador)
diarize_model = whisperx.DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)
diarize_segments = diarize_model(audio, num_speakers=numero_participantes)
result = whisperx.assign_word_speakers(diarize_segments, result)

#Lista de segmentos de audio con información de hablante y texto (y lo guardamos en un archivo .txt).
with open('transcription.txt', 'w', encoding='utf-8') as file:
  for segment in result["segments"]:
    if 'speaker' in segment:
      speaker = segment['speaker']
      text = segment['text'].strip()
      if text:
        file.write(f"{speaker}: {text}\n")

En caso contrario, ejecutar la siguiente línea de código

In [ ]:
#Asignar etiquetas de speaker (orador)
diarize_model = whisperx.DiarizationPipeline(use_auth_token=HF_TOKEN, device=device)
diarize_segments = diarize_model(audio)
result = whisperx.assign_word_speakers(diarize_segments, result)

#Lista de segmentos de audio con información de hablante y texto (y lo guardamos en un archivo .txt).
with open('transcription.txt', 'w', encoding='utf-8') as file:
  for segment in result["segments"]:
    if 'speaker' in segment:
      speaker = segment['speaker']
      text = segment['text'].strip()
      if text:
        file.write(f"{speaker}: {text}\n")

Al ejecutar la siguiente celda, podremos observar cómo se clasificó cada segmento de texto según los participantes (speakers) correspondientes.

In [ ]:
diarize_segments

Si desea descargar la transcripción diarizada como un archivo de texto, ejecutar la siguiente celda.

In [ ]:
files.download('transcription.txt')

Creamos un DataFrame con los resultados obtenidos

Un **DataFrame** de *pandas* es una forma de representar y trabajar con datos tabulares.

In [ ]:
audioDF = pd.DataFrame(result["segments"])
audioDF.head()

Con el fin de mejorar el AS y CE en el texto, agrupamos las filas consecutivas del mismo hablante en el DataFrame, consolidando el texto, los tiempos de inicio y fin, y la lista de palabras en una única oración por hablante.

In [ ]:
def group_consecutive_speakers(audioDF):
    """
    Groups consecutive speaker segments into single sentences, adjusting 'end' time and 'words'.
    """

    if not isinstance(audioDF, pd.DataFrame) or \
       not all(col in audioDF.columns for col in ['speaker', 'start', 'end', 'text', 'words']):
        raise ValueError("Input must be a pandas DataFrame with 'speaker', 'start', 'end', 'text', and 'words' columns.")

    grouped_segments = []
    current_speaker = None
    current_start = None
    current_end = None
    current_text = ""
    current_words = []

    for _, row in audioDF.iterrows():
        text = row['text'].strip()
        if row['speaker'] != current_speaker:
            if current_speaker is not None:
                grouped_segments.append({
                    'speaker': current_speaker,
                    'start': current_start,
                    'end': current_end,
                    'text': current_text.strip(),
                    'words': current_words
                })
            current_speaker = row['speaker']
            current_start = row['start']
            current_end = row['end']
            current_text = text
            current_words = row['words']
        else:
            current_end = row['end']
            current_text += " " + text
            current_words.extend(row['words'])

    # Agrega el último segmento
    if current_speaker is not None:
        grouped_segments.append({
            'speaker': current_speaker,
            'start': current_start,
            'end': current_end,
            'text': current_text.strip(),
            'words': current_words
        })

    return pd.DataFrame(grouped_segments)

In [ ]:
audioDF = group_consecutive_speakers(audioDF)
audioDF.head()

Si desea descargar el **dataframe** como un archivo .csv, ejecutar la siguiente celda

In [ ]:
# Guardar el dataframe como un archivo CSV
audioDF.to_csv('transcription_segments.csv', index=False)
# Descargar el archivo CSV
files.download('transcription_segments.csv')

## Etiquetar Participantes



En el DataFrame **audioDF**, podemos ver que los participantes están clasificados con etiquetas como "SPEAKER_00", "SPEAKER_01", etc.

Para reemplazar estas etiquetas por los nombres reales de los participantes, primero ejecutaremos la siguiente celda, que generará un fragmento de audio correspondiente a cada hablante, facilitando su identificación auditiva.

**Aclaración:** Si se desea renombrar a los speakers, es necesario ejecutar toda esta sección.

In [ ]:
# Set para registrar oradores ya procesados
processed_speakers = set()
# Lista para almacenar los oradores en orden de procesamiento
orators = []

# Cargar el archivo de audio completo
audio = AudioSegment.from_file(filename)

# Crear lista temporal: índice, orador, inicio, fin, duración
temp_list = [
    (idx, row['speaker'], row['start'], row['end'], row['end'] - row['start'])
    for idx, row in audioDF.iterrows()
]

# Buscar el primer segmento > 2 segundos por orador
seen = set()
filtered_segments = []
for idx, speaker, start, end, duration in temp_list:
    if speaker not in seen and duration > 2: #Modificar si se desea
        seen.add(speaker)
        filtered_segments.append((idx, speaker, start, end, duration))

# Procesar y reproducir los segmentos
for idx, speaker, start, end, duration in filtered_segments:
    if speaker not in processed_speakers:
        processed_speakers.add(speaker)

        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        segment = audio[start_ms:end_ms]
        segment.export(f"temp_segment_{speaker}.wav", format="wav")

        print(f"\nReproduciendo audio para {speaker}...")
        display(Audio(f"temp_segment_{speaker}.wav"))

        orators.append(speaker)


Una vez que hemos identificado a los participantes a través de los audios, ejecuta la siguiente celda y completá según corresponda.

Para guardar los cambios, presiona **Enter**.

In [ ]:
orator_names = {}

for speaker in orators:
    # Solicitar el nombre del orador al usuario
    speaker_name = input(f"Ingrese el nombre del orador para {speaker}: ")

    # Guardar el nombre del orador en un diccionario
    orator_names[speaker] = speaker_name

# Actualizar el DataFrame con los nuevos nombres de los oradores
for index, row in audioDF.iterrows():
    speaker = row['speaker']

    if speaker in orator_names:
        # Cambiar el nombre del orador en el DataFrame
        audioDF.at[index, 'speaker'] = orator_names[speaker]

Confirmamos que el cambio se haya realizado correctamente.

In [ ]:
audioDF.head()

## Filtrar StopWords

Las stopwords son palabras de uso frecuente, como "el", "la", "un", "una" o "en", que suelen ser omitidas en tareas de procesamiento de lenguaje natural, ya que aportan poca información semántica relevante para el análisis textual o la recuperación de información.

Además, se descartan las oraciones que contienen menos de 4 tokens, quedándonos únicamente con aquellas que tienen 4 o más tokens para asegurar un análisis más significativo.



In [ ]:
# Definir stopwords
stop_words = set(stopwords.words('spanish'))

# Puntuación a eliminar (excepto signos de exclamación/interrogación)
punctuation_to_strip = set(string.punctuation) | {"«", "»", "…"}
punctuation_to_strip -= {"¡", "¿", "!", "?"}

def remove_stopwords_es(text):
    # Tokenizamos
    tokens = word_tokenize(text.lower(), language='spanish')

    cleaned_tokens = []
    for token in tokens:
        # Quitar puntuación innecesaria
        word_clean = token.strip("".join(punctuation_to_strip))

        # Conservar si no es stopword ni vacío
        if word_clean and word_clean not in stop_words:
            cleaned_tokens.append(word_clean)

    # Reconstruir texto y unir signos sueltos a la palabra anterior
    texto = ' '.join(cleaned_tokens)

    # Quitar espacios antes de los signos de admiración/interrogación
    texto = re.sub(r'\s+([?!¡¿])', r'\1', texto)

    return texto

In [ ]:
audioDF['duration'] = audioDF['end'] - audioDF['start']

audioDF["text_clean"] = audioDF["text"].apply(remove_stopwords_es)
audioDF = audioDF[audioDF['text_clean'].str.strip().astype(bool)]
audioDF = audioDF[audioDF['text_clean'].apply(lambda x: len(str(x).split()) >= 4)]
audioDF.head()

## AS y CE

En la siguiente casilla podrá indicar el idioma del audio/video a analizar.

Nuestro modelo está especialmente diseñado para realizar el análisis en **español**, que es el idioma configurado por defecto.

In [ ]:
lenguaje = "es" # Idioma del audio a analizar. Para modificarlo, cambie lo que está dentro de las comillas por "es" (sin las comillas) cuando el idioma es español o "en" cuando es en inglés

La siguiente celda va a realizar el **análisis de sentimientos y de emociones** a cada frase que se ha transcripto anteriormente gracias a WhisperX.

In [ ]:
sentiment_analyzer = create_analyzer(task="sentiment", lang=lenguaje)
emotion_analyzer = create_analyzer(task="emotion", lang=lenguaje)

audioDF['sentimiento'] = audioDF['text_clean'].apply(lambda text: sentiment_analyzer.predict(text).output)
audioDF['emocion'] = audioDF['text_clean'].apply(lambda text: emotion_analyzer.predict(text).output)

audioDF['sentimiento_score'] = audioDF['text_clean'].apply(lambda text: sentiment_analyzer.predict(text).probas)
audioDF['emocion_score'] = audioDF['text_clean'].apply(lambda text: emotion_analyzer.predict(text).probas)

De esta manera, gracias a Pysentimiento, tenemos en nuestro DataFrame cada frase con su **sentimiento** y su **emoción** respectivamente.

En la siguiente celda podremos observar los sentimientos y emociones por interacción de cada participante

In [ ]:
audioDF[['text','speaker','sentimiento','emocion']]

## Análisis complementario en inglés

Para optimizar el AS y CE, se traducirán al inglés los textos que **PySentimiento** haya clasificado con la emoción *'others'* o el sentimiento *'NEU'*, y se aplicará un análisis complementario utilizando los modelos **PySentimiento** en inglés y **roberta-base-go_emotions**.

La ventaja principal de este último modelo radica en su capacidad para clasificar hasta 26 emociones, en contraste con *PySentimiento*, que se limita a las 6 emociones básicas.

**Importante**: ejecutar esta sección solo si el DataFrame **auxDF** no está vacío.

In [ ]:
# Traducimos solo los textos cuya emocion es 'others' o el sentimiento 'neutral'
# Para no perder datos, usamos un DataFrame auxiliar con esos casos

auxDF = audioDF[(audioDF['emocion'] == 'others') | (audioDF['sentimiento'] == 'NEU')]
auxDF

###**Traducción con Deep-Translator**

Esta herramienta ofrece compatibilidad con múltiples motores de traducción. En este caso, se optó por utilizar **GoogleTranslator**

In [ ]:
def traducir(texto):
    try:
        return GoogleTranslator(source='auto', target='en').translate(texto)
    except Exception as e:
        print(f"Error traduciendo: {texto} -> {e}")
        return None

In [ ]:
auxDF['text_translated'] = auxDF['text'].apply(traducir)
auxDF.head()

###Filtrar Stopwords

Se eliminan las **stopwords** del texto previamente traducido al inglés con el objetivo de reducir el ruido y mejorar la calidad del análisis semántico.

Se conserva el umbral mínimo de **4 tokens** tras la traducción y el filtrado, a fin de mantener la coherencia metodológica y la estabilidad de los resultados.


In [ ]:
# Definir stopwords
stop_words = set(stopwords.words('english'))

# Puntuación a eliminar (excepto signos de exclamación/interrogación)
punctuation_to_strip = set(string.punctuation) | {"«", "»", "…"}
punctuation_to_strip -= {"¡", "¿", "!", "?"}

def remove_stopwords_en(text):
    # Tokenizamos
    tokens = word_tokenize(text.lower(), language='english')

    cleaned_tokens = []
    for token in tokens:
        # Quitar puntuación innecesaria
        word_clean = token.strip("".join(punctuation_to_strip))

        # Conservar si no es stopword ni vacío
        if word_clean and word_clean not in stop_words:
            cleaned_tokens.append(word_clean)

    # Reconstruir texto y unir signos sueltos a la palabra anterior
    texto = ' '.join(cleaned_tokens)

    # Quitar espacios antes de los signos de admiración/interrogación
    texto = re.sub(r'\s+([?!¡¿])', r'\1', texto)

    return texto

In [ ]:
auxDF["text_clean_en"] = auxDF["text_translated"].apply(remove_stopwords_en)
auxDF = auxDF[auxDF['text_clean_en'].apply(lambda x: len(str(x).split()) >= 4)]
auxDF.head()

###CE con roberta-base-go_emotions

Realizamos la CE de las frases traducidas con el modelo **"roberta-base-go_emotions"**

In [ ]:
def analisis_baseGoEmotions_samlowe(text):
    model_outputs = classifier([text])  # <- Usar el texto recibido
    return model_outputs[0][0]['label']  # Retorna la emoción con mayor score

In [ ]:
auxDF['emotion'] = auxDF['text_translated'].apply(analisis_baseGoEmotions_samlowe)
auxDF.head()

El modelo de CE utilizado está entrenado con el dataset **GoEmotions** de Google, el cual permite identificar una gama más amplia de emociones que las seis emociones básicas que devuelve **PySentimiento** en español. Para hacer comparables los resultados, aplicaremos el esquema de agrupación propuesto en el [paper de GoEmotions](https://arxiv.org/pdf/2005.00547), mapeando las emociones del modelo al conjunto de seis emociones básicas utilizadas por PySentimiento.



In [ ]:
mapping = {
    # anger group
    'anger': 'anger',
    'annoyance': 'anger',
    'disapproval': 'anger',

    # disgust group
    'disgust': 'disgust',

    # fear group
    'fear': 'fear',
    'nervousness': 'fear',

    # joy group
    'admiration': 'joy',
    'amusement': 'joy',
    'approval': 'joy',
    'caring': 'joy',
    'desire': 'joy',
    'excitement': 'joy',
    'gratitude': 'joy',
    'joy': 'joy',
    'love': 'joy',
    'pride': 'joy',
    'optimism': 'joy',
    'relief': 'joy',

    # sadness group
    'sadness': 'sadness',
    'disappointment': 'sadness',
    'embarrassment': 'sadness',
    'grief': 'sadness',
    'remorse': 'sadness',

    # surprise group
    'confusion': 'surprise',
    'curiosity': 'surprise',
    'realization': 'surprise',
    'surprise': 'surprise',

    # neutral group
    'neutral': 'neutral'
}

In [ ]:
auxDF.loc[:, 'emotion'] = auxDF['emotion'].map(mapping).fillna('neutral')
auxDF.head()

### Análisis de Sentimiento con PySentimiento en inglés

In [ ]:
lenguaje = "en" # Idioma del audio a analizar. Para modificarlo, cambie lo que está dentro de las comillas por "es" (sin las comillas) cuando el idioma es español o "en" cuando es en inglés

In [ ]:
sentiment_analyzer_en = create_analyzer(task="sentiment", lang=lenguaje)

auxDF['sentiment'] = auxDF['text_clean_en'].apply(lambda text: sentiment_analyzer_en.predict(text).output)
auxDF.head()

### Reemplazo de los sentimientos y emociones

Se reemplazan los valores de la columna "sentimiento" que son iguales a "NEU" por los correspondientes de la columna "sentiment". De manera análoga, los valores "others" en la columna "emocion" se sustituyen por los valores equivalentes presentes en la columna "emotion".

In [ ]:
# Reemplazos condicionales
auxDF.loc[auxDF["sentimiento"] == "NEU", "sentimiento"] = auxDF["sentiment"]
auxDF.loc[auxDF["emocion"] == "others", "emocion"] = auxDF["emotion"]

#Eliminamos las columnas
auxDF = auxDF.drop(['sentiment','emotion'], axis=1)

# Renombrar columnas
auxDF = auxDF.rename(columns={
    "sentimiento": "sentiment",
    "emocion": "emotion"
})

In [ ]:
auxDF.head()

Unimos los datos obtenidos en este análisis con los originales

In [ ]:
auxDF = auxDF[['text','sentiment','emotion']]

audioDF = audioDF.merge(auxDF, how='left', on='text')
audioDF

Reemplazamos los valores originales por los nuevos

In [ ]:
# Reemplazar 'sentimiento' solo si es 'NEU' y 'sentiment' no es NaN
audioDF.loc[
    (audioDF["sentimiento"] == "NEU") & audioDF["sentiment"].notna(),
    "sentimiento"
] = audioDF["sentiment"]

# Reemplazar 'emocion' solo si es 'others' y 'emotion' no es NaN
audioDF.loc[
    (audioDF["emocion"] == "others") & audioDF["emotion"].notna(),
    "emocion"
] = audioDF["emotion"]

audioDF.loc[audioDF["emocion"] == "others", "emocion"] = "neutral"
audioDF = audioDF.drop(['sentiment','emotion'], axis=1)

audioDF

##Gráficos

In [ ]:
#Traducir las emociones
audioDF['emocion'] = audioDF['emocion'].replace({
    'joy': 'alegría',
    'surprise': 'sorpresa',
    'disgust': 'disgusto',
    'sadness': 'tristeza',
    'fear': 'miedo',
    'anger': 'enojo'
})

colores_sentimientos = {
    'POS': '#00A651',
    'NEG': '#F05A61',
    'NEU':  '#FFCB08'
}

colores_emociones = {
    'enojo': '#F05A61',
    'alegría': '#FFCB08',
    'tristeza': '#2884C6',
    'miedo': '#00A651',
    'sorpresa': '#009ACE',
    'disgusto':  '#8A73B3',
    'neutral':  '#F7923D',
    'others':  '#F7923D'
}

###Gráfico de duración por orador

In [ ]:
#Grafica la duración por orador
speaker_durations = audioDF.groupby('speaker')['start'].sum()
plt.figure(figsize=(10, 6))
plt.bar(speaker_durations.index, speaker_durations.values)
plt.xlabel('Orador')
plt.ylabel('Duración total (segundos)')
plt.title('Duración de la conversación por orador')
plt.xticks(rotation=45, ha='right')
plt.show()

###Gráfico de sentimientos por orador

In [ ]:
sentiment_durations = audioDF.groupby(["speaker", "sentimiento"])["duration"].sum().unstack(fill_value=0)

# Calcular porcentajes por orador
sentiments_percentages = sentiment_durations.div(sentiment_durations.sum(axis=1), axis=0) * 100

# Graficar barras apiladas
ax = sentiments_percentages.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6),
    color=[colores_sentimientos[col] for col in sentiment_durations.columns]
)

# Personalización
plt.xlabel('Orador')
plt.ylabel('Porcentaje (%)')
plt.title('Distribución de Sentimientos por Orador (según tiempo hablado)')
plt.xticks(rotation=45)
plt.legend(title="Sentimientos", bbox_to_anchor=(1.05, 1), loc='upper left')

# Agregar etiquetas a la derecha de cada bloque
for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(f'{height:.1f}%',
                    (p.get_x() + p.get_width(), p.get_y() + height / 2),
                    ha='left', va='center', fontsize=10, color='black')

plt.tight_layout()
plt.savefig("sentimiento_predominante_orador.png", format="png", bbox_inches="tight")
plt.show()

###Gráfico de emociones por orador

In [ ]:
emotions_durations = audioDF.groupby(["speaker", "emocion"])["duration"].sum().unstack(fill_value=0)
emotions_percentages = emotions_durations.div(emotions_durations.sum(axis=1), axis=0) * 100

ax = emotions_percentages.plot(
    kind='bar',
    stacked=True,
    figsize=(10, 6),
    color=[colores_emociones[col] for col in emotions_durations.columns]
)

# Personalización
plt.xlabel('Orador')
plt.ylabel('Porcentaje (%)')
plt.title('Distribución de Emociones por Orador (según tiempo hablado)')
plt.xticks(rotation=45)
plt.legend(title="Emocion", bbox_to_anchor=(1.05, 1), loc='upper left')

# Etiquetas a la derecha de la barra
for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(f'{height:.1f}%',
                    (p.get_x() + p.get_width(), p.get_y() + height / 2),
                    ha='left', va='center', fontsize=10, color='black')

plt.tight_layout()
plt.savefig("emocion_predominante_orador.png", format="png", bbox_inches="tight")
plt.show()

###Grafico de sentimientos durante toda la interacción

In [ ]:
sentiment_durations = audioDF.groupby("sentimiento")["duration"].sum()
sentiment_percentages = (sentiment_durations / sentiment_durations.sum()) * 100

colors = [colores_sentimientos.get(sent, "#CCCCCC") for sent in sentiment_percentages.index]

plt.figure(figsize=(8, 8))
plt.pie(sentiment_percentages, labels=sentiment_percentages.index, autopct='%1.1f%%', startangle=140, colors=colors)

plt.title("Distribución de Sentimientos en toda la interacción")

plt.tight_layout()
plt.savefig("sentimiento_predominante_toda_interaccion.png", format="png", bbox_inches="tight")
plt.show()

###Gráfico de emociones durante toda la interacción

In [ ]:
emotions_durations = audioDF.groupby("emocion")["duration"].sum()
emotions_percentages = (emotions_durations / emotions_durations.sum()) * 100

colors = [colores_emociones.get(sent, "#CCCCCC") for sent in emotions_percentages.index]

plt.figure(figsize=(8, 8))
plt.pie(emotions_percentages, labels=emotions_percentages.index, autopct='%1.1f%%', startangle=140, colors=colors)

plt.title("Distribución de Emociones en toda la interacción")

plt.tight_layout()
plt.savefig("emocion_predominante_toda_interaccion.png", format="png", bbox_inches="tight")
plt.show()

###Evolución de los sentimientos a lo largo del tiempo

In [ ]:
sentimiento_map = {'NEG': -1, 'NEU': 0, 'POS': 1}
audioDF['sentimiento_num'] = audioDF['sentimiento'].map(sentimiento_map)
audioDF['mid_time'] = (audioDF['start'] + audioDF['end']) / 2

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(data=audioDF, x='mid_time', y='sentimiento_num', hue='speaker', marker='o')
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title('Sentimiento a lo largo del tiempo por participante')
plt.xlabel('Tiempo (s)')
plt.ylabel('Sentimiento')
plt.yticks([-1, 0, 1], ['Negativo', 'Neutral', 'Positivo'])
plt.legend(title='Orador')
plt.grid(True)
plt.tight_layout()

plt.tight_layout()
plt.savefig("sentimiento_tiempo_speakers.png", format="png", bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(12, 4))
sns.lineplot(data=audioDF.sort_values('mid_time'), x='mid_time', y='sentimiento_num', marker='o', color='black')
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title('Sentimiento a lo largo del tiempo (interacción completa)')
plt.xlabel('Tiempo (s)')
plt.ylabel('Sentimiento')
plt.yticks([-1, 0, 1], ['Negativo', 'Neutral', 'Positivo'])
plt.grid(True)
plt.tight_layout()

# Ajustar el diseño para que todo sea visible
plt.tight_layout()
plt.savefig("sentimiento_tiempo.png", format="png", bbox_inches="tight")
plt.show()

###Nube de palabras

In [ ]:
text = " ".join(audioDF['text_clean'].astype(str))

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')

plt.tight_layout()
plt.savefig("wordcloud.png", format="png", bbox_inches="tight")
plt.show()

###Palabras más frecuentes

In [ ]:
text = " ".join(audioDF['text_clean'].astype(str)).lower()

words = re.findall(r'\b\w+\b', text)

word_counts = Counter(words)

most_common_words = word_counts.most_common(20)

plt.figure(figsize=(10, 6))
plt.bar(*zip(*most_common_words))
plt.xlabel("Palabras")
plt.ylabel("Frecuencia")
plt.title("Las 20 palabras más frecuentes")
plt.xticks(rotation=45, ha='right')

plt.savefig("palabras_frecuentes.png", format="png", bbox_inches="tight")
plt.show()

In [ ]:
audioDF = audioDF.rename(columns={
    "speaker": "orador",
    "duration": "duracion (seg)",
    "text": "texto"
})

## Reporte final

In [ ]:
# Instalar dependencias necesarias
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!apt-get install -y wkhtmltopdf
!pip install pdfkit

import pdfkit
from google.colab import files
import base64

In [ ]:
import base64
import pdfkit
from google.colab import files
import pandas as pd

# Función para convertir imagen a base64
def img_to_base64(img_path):
    with open(img_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Configurar la ruta de wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

html_audioDF = audioDF[["orador","duracion (seg)","texto","emocion","sentimiento"]].to_html(index=False)

# Lista de imágenes esperadas
imagenes = [
    "emocion_predominante_orador.png",
    "sentimiento_predominante_orador.png",
    "emocion_predominante_toda_interaccion.png",
    "sentimiento_predominante_toda_interaccion.png",
    "wordcloud.png",
    "palabras_frecuentes.png",
    "sentimiento_tiempo_speakers.png",
    "sentimiento_tiempo.png"
]

# Convertir a base64
imagenes_base64 = {img: img_to_base64(img) for img in imagenes}

# HTML Final
html_content = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Analizador Automático de Sentimientos - Informe</title>
    <style>
        .page-break {{
            page-break-before: always;
        }}

        body {{
            font-family: 'Arial', sans-serif;
            margin: 40px;
        }}

        h1 {{
            color: #18181b;
            font-size: 1.5rem;
            text-align: center;
            margin-bottom: 30px;
        }}

        .flex-row {{
            display: flex;
            flex-direction: row;
            justify-content: space-between;
            margin-bottom: 20px;
        }}

        .flex-col {{
            display: flex;
            flex-direction: column;
            gap: 20px;
            align-items: center;
        }}

        .graphs-row {{
            max-width: 48%;
            height: auto;
        }}

        .graphs-col {{
            max-width: 70%;
            height: auto;
        }}

        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            font-size: 10px;  /* Reducir el tamaño de la fuente */
        }}
        th, td {{
            padding: 6px;  /* Reducir el padding */
            text-align: left;
            border: 1px solid #ccc;
        }}
        th {{
            background-color: #f4f4f4;
        }}
    </style>
</head>
<body>
    <h1>Analizador de Sentimientos y Emociones - Informe</h1>

    <div class="flex-row">
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['sentimiento_predominante_orador.png']}"/>
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['emocion_predominante_orador.png']}"/>
    </div>

    <div class="flex-row">
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['sentimiento_predominante_toda_interaccion.png']}"/>
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['emocion_predominante_toda_interaccion.png']}"/>
    </div>

    <div class="flex-row">
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['wordcloud.png']}"/>
        <img class="graphs-row" src="data:image/png;base64,{imagenes_base64['palabras_frecuentes.png']}"/>
    </div>

    <div class="page-break"></div>

    <div class="flex-col">
        <img class="graphs-col" src="data:image/png;base64,{imagenes_base64['sentimiento_tiempo_speakers.png']}" />
        <img class="graphs-col" src="data:image/png;base64,{imagenes_base64['sentimiento_tiempo.png']}"/>
    </div>

    <h2 style="margin-top: 2rem;">Tabla de Datos Analizados</h2>
    <div style="width: 100%;">
        {html_audioDF}
    </div>
</body>
</html>
"""



# Configuración de wkhtmltopdf
config = pdfkit.configuration(wkhtmltopdf="/usr/bin/wkhtmltopdf")

# Guardar PDF
pdf_path = "/content/informe_AS-EC.pdf"
pdfkit.from_string(html_content, pdf_path, configuration=config)

# Descargar en Colab
files.download(pdf_path)